In [1]:
import pandas as pd
import random

In [2]:
# Veut dire que c’est un passage par adresse des 3 variables IX, IY et IZ.
# 3 variables entières qu’il faut initialiser avant le 1er appel de la fonction.
# Veut dire que la fonction est typée, elle retourne un nombre réel.
def alea(IX, IY, IZ):
    # Déclaration d’une variable inter de type réel
    inter = 0.0
    # A chaque exécution de la fonction, les valeurs des 3 variables sont recalculées, d’où un passage par adresse.
    # % : division entière
    # // : reste de la division entière
    IX = 171 * (IX % 177) - 2 * (IX // 177)
    IY = 172 * (IY % 176) - 35 * (IY // 176)
    IZ = 170 * (IZ % 178) - 63 * (IZ // 178)
    # S’il y a un dépassement de capacité sur une variable, on ajoute un nombre très grand, pour la machine 16 bits,
    # pour ramener à une valeur positive. Ce qui marche sur une machine 16 bits marche sur une 32 ou 64 bits.
    if IX < 0:
        IX = IX + 30269
    if IY < 0:
        IY = IY + 30307
    if IZ < 0:
        IZ = IZ + 30323
    inter = ((IX / 30269) + (IY / 30307) + (IZ / 30323))
    return [round(inter - int(inter), 4),IX, IY, IZ]

In [3]:
salaire = [(40000, 30000), (30000, 20000), (20000, 15000), (15000, 10000), (10000, 7500), (7500, 5000), (5000, 3000)]
frequenceSalaire = [5, 10, 10, 15, 20, 20, 20]
age = [(62, 53), (52, 41), (40, 31), (30, 21)]
frequenceAge = [30, 30, 20, 20]
ageEmbauche = [(21, 22), (23, 24), (25, 28), (29, 32), (33, 35), (36, 40)]
frequenceAgeEmbauche = [5, 20, 25, 25, 20, 5]
salaire_embauche=[(32000,24000),(24000,16000),(16000,12000),(12000,8000),(8000,6000),(6000,4000),(4000,3000)]
frequenceSalaireEmbauche=[5,5,10,20,20,20,20]
years = [24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
totEmp=10000
totRet=1000

In [4]:
def cumuleFrequenceIntervale(frequences):
    cumules = []
    index = 0
    for i in frequences:
        cumules.append([index, i+index])
        index += i
    return cumules

In [5]:
def indexFromTable(cumules, aleaNumber):
    i = 0
    for intervale in cumules :
        if intervale[1] > aleaNumber*100 :
            return i
        i+=1
    return i

In [6]:
Employe_2023 = pd.DataFrame(columns=["id", "age", "salaire"])
Retraite_2023 = pd.DataFrame(columns=["id", "age", "salaire"])

In [7]:
for index in range(11000):
    if index == 0 :
        IX = 100
        IY = 100
        IZ = 100
    else:
        IX = alea(IX, IY, IZ)[1]
        IY = alea(IX, IY, IZ)[2]
        IZ = alea(IX, IY, IZ)[3]
    salaireIndex = indexFromTable(cumuleFrequenceIntervale(frequenceSalaire), alea(IX, IY, IZ)[0])
    ageIndex = indexFromTable(cumuleFrequenceIntervale(frequenceAge), alea(IX, IY, IZ)[0])
    i = index+1
    s = random.randrange(salaire[salaireIndex][1], salaire[salaireIndex][0], 100)
    if index <10000:
        a = random.randrange(age[ageIndex][1], age[ageIndex][0], 1)
        new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s]})
        Employe_2023 = pd.concat([Employe_2023, new_row], ignore_index=True)
    else:
        a = random.randint(66, 85)
        new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s]})
        Retraite_2023 = pd.concat([Retraite_2023, new_row], ignore_index=True)

In [8]:
Employe_2023.head()

,id,age,salaire
0,1,38,5600
1,2,48,7500
2,3,57,29500
3,4,21,4300
4,5,26,4900


In [9]:
Retraite_2023.head()

,id,age,salaire
0,10001,73,5900
1,10002,84,3600
2,10003,79,20100
3,10004,78,18300
4,10005,82,5900


In [10]:
def pensionnaireFive(dfRetrait) :
    pensionnaire65 = pd.DataFrame(columns=["id", "age", "salaire", "pension"])
    for index, row in dfRetrait.iterrows():
        a = row["age"]
        s = row["salaire"]
        p = ((44*2)/100)*row["salaire"]
        new_row = pd.DataFrame( {"id" : row["id"], "age": [a], "salaire": [s], "pension": [p]})
        pensionnaire65 = pd.concat([pensionnaire65, new_row], ignore_index=True)
    return pensionnaire65

In [11]:
pensionnaireFive(Retraite_2023).head()

,id,age,salaire,pension
0,10001,73,5900,5192.0
1,10002,84,3600,3168.0
2,10003,79,20100,17688.0
3,10004,78,18300,16104.0
4,10005,82,5900,5192.0


In [32]:
def pensionnaireTwo(dfEmploye, dfRetraite):
    pensionnaire62 = pd.DataFrame(columns=["id", "age", "salaire", "pension"])
    pensionnaire65 = pensionnaireFive(dfRetraite)
    pensionnaire62 = pd.concat([pensionnaire62, pensionnaire65], ignore_index=True)
    for index, row in dfEmploye.iterrows():
        if row["age"]>62:
            a = row["age"]
            s = row["salaire"]
            p = ((44*2)/100)*row["salaire"]
            new_row = pd.DataFrame( {"id" : row["id"], "age": [a], "salaire": [s], "pension": [p]})
            pensionnaire62 = pd.concat([pensionnaire62, new_row], ignore_index=True)
    return pensionnaire62

In [13]:
pensionnaireTwo(Employe_2023, Retraite_2023).tail()

,id,age,salaire,pension
995,10996,75,9900,8712.0
996,10997,76,37100,32648.0
997,10998,71,4500,3960.0
998,10999,68,8500,7480.0
999,11000,81,24600,21648.0


In [14]:
def newRecrutement():
    Nouveux_Recrus = pd.DataFrame(columns=["id", "age", "salaire"])
    IX = 100
    IY = 100
    IZ = 100
    a=alea(IX, IY, IZ)[0]
    n = ((400 - 250) * a) + 250
    for index in range(int(n)):
        if index == 0 :
            IX = 100
            IY = 100
            IZ = 100
        else:
            IX = alea(IX, IY, IZ)[1]
            IY = alea(IX, IY, IZ)[2]
            IZ = alea(IX, IY, IZ)[3]
        salaireIndex = indexFromTable(cumuleFrequenceIntervale(frequenceSalaireEmbauche), alea(IX, IY, IZ)[0])
        ageIndex = indexFromTable(cumuleFrequenceIntervale(frequenceAgeEmbauche), alea(IX, IY, IZ)[0])
        i = index+1
        s = random.randrange(salaire_embauche[salaireIndex][1], salaire_embauche[salaireIndex][0], 100)
        a = random.randrange(ageEmbauche[ageIndex][0], ageEmbauche[ageIndex][1], 1)
        new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s]})
        Nouveux_Recrus = pd.concat([Nouveux_Recrus, new_row], ignore_index=True)
    return Nouveux_Recrus

In [15]:
data_frames = {}
data_frames['Employe_2023'] = Employe_2023
data_frames['Retraite_2023'] = Retraite_2023
for i in range(24, 34) :
    data_frames['Employe_20'+str(i)] = pd.DataFrame(columns=["id", "age", "salaire"])
    data_frames['Retraite_20'+str(i)] = pd.DataFrame(columns=["id", "age", "salaire"])

In [16]:
def CalculEmployeRetraite(i):
    EmployeBeta = data_frames['Employe_20'+str(i-1)]
    RetraiteBeta = data_frames['Retraite_20'+str(i-1)]
    EmployeAlpha = data_frames['Employe_20'+str(i)]
    RetraiteAlpha = data_frames['Retraite_20'+str(i)]
    for index, row in EmployeBeta.iterrows():
        if row["age"] < 65:
            if i == 25 or i == 30:
                s = row["salaire"]*1.05
            else :
                s = row["salaire"]
            a = row["age"]+1
            new_row = pd.DataFrame({"id" : row["id"], "age": [a], "salaire": [s]})
            EmployeAlpha = pd.concat([EmployeAlpha, new_row], ignore_index=True)
        else :
            if i == 25 or i == 30:
                s = row["salaire"]*1.05
            else :
                s = row["salaire"]
            a = row["age"]+1
            new_row = pd.DataFrame({"id" : row["id"], "age": [a], "salaire": [s]})
            RetraiteAlpha = pd.concat([RetraiteAlpha, new_row], ignore_index=True)
    for index, row in RetraiteBeta.iterrows():
        new_row = pd.DataFrame({"id" : [row["id"]], "age": [row["age"]+1], "salaire": [row["salaire"]]})
        RetraiteAlpha = pd.concat([RetraiteAlpha, new_row], ignore_index=True)
    newRecru = newRecrutement()
    for index, row in newRecru.iterrows():
        new_row = pd.DataFrame({"id" : [row["id"]], "age": [row["age"]], "salaire": [row["salaire"]]})
        EmployeAlpha = pd.concat([EmployeAlpha, new_row], ignore_index=True)
    data_frames['Employe_20'+str(i)] = EmployeAlpha
    data_frames['Retraite_20'+str(i)] = RetraiteAlpha

In [17]:
for year in years :
    CalculEmployeRetraite(year)

In [19]:
data_frames["Retraite_2033"]

,id,age,salaire
0,33,66,13450.5
1,56,66,18301.5
2,86,66,37044.0
3,107,66,35059.5
4,259,66,42997.5
...,...,...,...
2972,10996,85,9900.0
2973,10997,86,37100.0
2974,10998,81,4500.0
2975,10999,78,8500.0


In [26]:
pensionnaireFive(data_frames["Retraite_2024"])

,id,age,salaire,pension
0,10001,74,5900,5192.0
1,10002,85,3600,3168.0
2,10003,80,20100,17688.0
3,10004,79,18300,16104.0
4,10005,83,5900,5192.0
...,...,...,...,...
995,10996,76,9900,8712.0
996,10997,77,37100,32648.0
997,10998,72,4500,3960.0
998,10999,69,8500,7480.0


In [34]:
pensionnaireTwo(data_frames["Employe_2033"], data_frames["Retraite_2033"])

,id,age,salaire,pension
0,33,66,13450.50,11836.44
1,56,66,18301.50,16105.32
2,86,66,37044.00,32598.72
3,107,66,35059.50,30852.36
4,259,66,42997.50,37837.80
...,...,...,...,...
3979,9924,64,26239.50,23090.76
3980,9926,63,28334.25,24934.14
3981,9931,64,39690.00,34927.20
3982,9949,65,11686.50,10284.12
